In [1]:
import os
os.chdir('..')

from puzzle_generator import *

In [2]:
width = 8
height = 8
puzzle = RectangularPuzzle(
    width=width,
    height=height
)

puzzle.solving_timeout = 300

puzzle.naming = {
    None: ' ',
    'e': '_',
}

puzzle.domain_program += """
    1 { guessed_number(num_hit_targets,4..6) } 1.
    1 { guessed_number(num_safe_targets,4..6) } 1.
    1 { guessed_number(num_missile_sites,4..9) } 1.
    1 { guessed_number(num_defense_sites,4..9) } 1.
    1 { guessed_number(num_obstacles,0) } 1.
    
    value(e).
    site(h;s;m;d;o).
    value(S) :- site(S).
"""

puzzle.puzzle_gen_program += """
    { puzzle(C,V) : solution(C,V) }.
"""
puzzle.puzzle_constraints_program += """
    #minimize { 1,puzzle(C,V) : puzzle(C,V) }.
    :- puzzle(C,e).
"""

puzzle.solution_program = """
    1 { solution(C,V) : value(V) } 1 :- cell(C).
    solution(C,V) :- puzzle(C,V).
"""
puzzle.solution_program += enc_library['ring_around_cell']
puzzle.solution_program += enc_library['adjacent_cells']
puzzle.solution_program += """
    :- adjacent_cells(C1,C2), C1 != C2,
        solution(C1,V1), solution(C2,V2), site(V1), site(V2).
    :- solution(C1,m), ring_around_cell(C1,C2,1), C1 != C2,
        solution(C2,S), site(S).

    in_range(C1,C2) :-
        solution(C1,m), solution(C2,(h;s)),
        ring_around_cell(C1,C2,D), D <= 2.
    in_range(C1,C2) :-
        solution(C1,d), solution(C2,(h;s)),
        ring_around_cell(C1,C2,D), D <= 1.
    
    active(C1,m) :- solution(C1,m), in_range(C1,C2).
    1 { targets(C1,C2) : in_range(C1,C2) } 1 :- active(C1,m).
    1 { protects(C1,C2) : in_range(C1,C2) } 1 :- solution(C1,d).
    
    :- solution(C1,d),
        not in_range(C1,C2) : solution(C2,h);
        not in_range(C1,C2) : solution(C2,s).
    
    :- solution(C1,h),
        A = #count { C2 : targets(C2,C1) },
        P = #count { C2 : protects(C2,C1) },
        A <= P.
    :- solution(C1,s),
        A = #count { C2 : targets(C2,C1) },
        P = #count { C2 : protects(C2,C1) },
        A > P.

    :- guessed_number(num_hit_targets,N),
        not N { solution(C,h) } N.
    :- guessed_number(num_safe_targets,N),
        not N { solution(C,s) } N.
    :- guessed_number(num_missile_sites,N),
        not N { solution(C,m) } N.
    :- guessed_number(num_defense_sites,N),
        not N { solution(C,d) } N.
    :- guessed_number(num_obstacles,N),
        not N { solution(C,o) } N.
"""

In [3]:
puzzle.generate(
    verbose=True,
    precompute_solution=False,
)

Done grounding..
...........
Stopped after solving timeout..
Solving time: 376.93 seconds



In [4]:
print(puzzle.pretty_repr_puzzle())

| | |s| | | | |m|
| | | | | | | | |
| | |h| | | |h| |
| | | | | |h| |s|
| | | | | | |s| |
| | | | |h| | |s|
| | | | | | |s| |
| | | | | |h| | |

num_obstacles = 0
num_defense_sites = 4
num_missile_sites = 5
num_safe_targets = 6
num_hit_targets = 5


In [5]:
print(puzzle.pretty_repr_solution())

|_|_|s|_|_|_|_|m|
|_|d|_|_|m|_|_|_|
|_|_|h|_|_|_|h|_|
|m|_|_|_|_|h|_|s|
|_|_|m|_|_|_|s|_|
|d|_|_|_|h|_|_|s|
|_|s|_|_|_|_|s|_|
|d|_|_|m|_|h|_|d|
